In [ ]:
! pip install python-dotenv
! pip install transformers torch huggingface_hub > /dev/null
! pip install torch torchvision > /dev/null
! pip install transformers datasets 
! pip install accelerate>=0.26.0

In [ ]:
# import pandas as pd 

# df=pd.read_csv("fol_expressions_20000.csv")
# column_titles=['english_description','expression']
# df=df.reindex(columns=column_titles)
# df.to_csv('fol_dataset.csv')

In [ ]:
import os 
from dotenv import load_dotenv

TOKEN = os.getenv("TOKEN")
TOKENIZERS_PARALLELISM = os.getenv("TOKENIZERS_PARALLELISM")
from huggingface_hub import login
login(token = TOKEN)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-1b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-3-1b-it")

In [ ]:
from datasets import load_dataset 
dataset = load_dataset('csv', data_files = "fol_dataset.csv")['train']
split_dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_set = split_dataset['train']
test_set = split_dataset['test']

In [ ]:
print(dataset.column_names)

In [ ]:
def preprocess(batch):
     inputs = tokenizer(
         batch['expression'],
         max_length=128,
         truncation=True,
         padding = 'max_length',
         
     )    
    
     labels = tokenizer(
         batch['english_description'],
         max_length =128,
         truncation = True, 
         padding = 'max_length'
     )
     inputs['labels']=labels['input_ids']
     return inputs

In [ ]:
train_tokenized = train_set.map(preprocess, batched=True)
test_tokenized = test_set.map(preprocess, batched=True)

In [ ]:
#setting the trainer 
from transformers import Trainer, TrainingArguments 


training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    learning_rate=2e-5,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
)

trainer.train()